# Objective

Many agronomic terms appear in natural language in multiple forms, e.g.:
* "The awns are rough", "It has rough awns", or "It is rough-awned". In all these cases, the plant part (PLAN), awn, is modified by an adjective, rough. The combination, "rough" + "awn" is a trait (TRAT).
* "early maturing", "matures early". In these cases, a trait (TRAT), 'maturing' is modified by an adjective, early. This combination "early" + "maturing" is a compound trait (TRAT).

In this notebook, we will run a small section of text against a trained NLP model, read the predictions, identify compoud traits based on the above rules, and output modified named entities in JSON format that include the compound traits.

In [1]:
import spacy


# Do a quick training test

In [1]:
from nerTraining import *

output_dir="NerModelTest"
training_file = 'Data/DavisLJ11/barley_p30_td.json'
n_iter = 100
trainModel(None,training_file, output_dir,n_iter)

Created pre-trained 'en' model
Training 25% done
Training 50% done
Training 75% done
Training complete!
Saved model to NerModelTest


# NLP parse some sample text

In [11]:
import PyPDF2
from spacy import displacy

# nlp = spacy.load('en_core_web_sm')
nlp = spacy.load('NerModelTest')

test_text = '''Kold is a six-rowed winter feed barley. It was released by the Oregon and Idaho AESs in 1993. It has rough awns and the aleurone is white. It is early maturing, and is high yielding.'''

doc = nlp(test_text)

colors = {'ALAS':'BlueViolet','CROP': 'Aqua','CVAR':'Chartreuse','PATH':'red','PED':'orange','PLAN':'pink','PPTD':'brown','TRAT':'yellow'}
cust_options = {'ents': ['ALAS','CROP','CVAR','PATH','PED','PLAN','PPTD','TRAT'], 'colors':colors}

displacy.render(doc, style='ent', jupyter=True, options=cust_options)


# Identify compound traits ADJ + PLAN = TRAT
## first flag entities and POS

In [12]:
print("Entities:")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
    
# Comment out the following due to lack of statistical training for NERModel
# print("\nNoun Chunks:")
# for chunk in doc.noun_chunks:
#     print(chunk.text, chunk.root.text, chunk.root.dep_,
#             chunk.root.head.text)
    
print("\nParts of Speech:")
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_)

Entities:
Kold 0 4 CVAR
six-rowed 10 19 TRAT
winter 20 26 TRAT
feed 27 31 TRAT
barley 32 38 CROP
Oregon 63 69 CVAR
Idaho 74 79 GPE
1993 88 92 DATE
awns 107 111 PLAN
aleurone 120 128 PLAN
maturing 151 159 TRAT

Parts of Speech:
Kold Kold PROPN NNP nsubj
is be AUX VBZ ROOT
a a DET DT det
six six NUM CD advmod
- - PUNCT HYPH punct
rowed row VERB VBN amod
winter winter NOUN NN compound
feed feed NOUN NN compound
barley barley NOUN NN attr
. . PUNCT . punct
It -PRON- PRON PRP nsubjpass
was be AUX VBD auxpass
released release VERB VBN ROOT
by by ADP IN agent
the the DET DT det
Oregon Oregon PROPN NNP nmod
and and CCONJ CC cc
Idaho Idaho PROPN NNP conj
AESs AESs PROPN NNPS pobj
in in ADP IN prep
1993 1993 NUM CD pobj
. . PUNCT . punct
It -PRON- PRON PRP nsubj
has have AUX VBZ ROOT
rough rough ADJ JJ amod
awns awn NOUN NNS dobj
and and CCONJ CC cc
the the DET DT det
aleurone aleurone NOUN NN nsubj
is be AUX VBZ conj
white white ADJ JJ acomp
. . PUNCT . punct
It -PRON- PRON PRP nsubj
is be AUX 

## Next, identify clauses fitting ADJ + PLAN = TRAT

from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)
pattern = [{"ENT_TYPE": "PLAN"}, {"POS": "NOUN"}, {"DEP": "dobj"}]
matcher.add("Plan2Trait", None, pattern)
matches = matcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

OK, that didn't work. Let's try another way

In [13]:
from spacy.tokens import Span

def compound_trait_entities(doc):
    new_ents = []
    for ent in doc.ents:
        # Only check for PLAN entities with a preceding one-token adjective
        # This needs to be generalized to explicitly include any dependency relationehip
        # (e.g., 'the awns are rough' in addition to 'rough awns')
        if ent.label_ == "PLAN" and ent.start != 0:
            prev_token = doc[ent.start - 1]
            print('DEBUG: ', ent.text, ent.start, ent.label_, prev_token.text, prev_token.pos_, prev_token.dep_)
            if prev_token.pos_ == 'ADJ' and prev_token.dep_ == 'amod':
                new_ent = Span(doc, ent.start - 1, ent.end, label='TRAT')
                new_ents.append(new_ent)
            else:
                new_ents.append(ent)
        else:
            new_ents.append(ent)
    doc.ents = new_ents
    return doc

new_doc = compound_trait_entities(doc)
print(new_doc)
print("Entities:")
for ent in new_doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


DEBUG:  awns 25 PLAN rough ADJ amod
DEBUG:  aleurone 28 PLAN the DET det
Kold is a six-rowed winter feed barley. It was released by the Oregon and Idaho AESs in 1993. It has rough awns and the aleurone is white. It is early maturing, and is high yielding.
Entities:
Kold 0 4 CVAR
six-rowed 10 19 TRAT
winter 20 26 TRAT
feed 27 31 TRAT
barley 32 38 CROP
Oregon 63 69 CVAR
Idaho 74 79 GPE
1993 88 92 DATE
rough awns 101 111 TRAT
aleurone 120 128 PLAN
maturing 151 159 TRAT


## Add the above as a pipeline

In [ ]:
# Add the component after the named entity recognizer
nlp.add_pipe(compound_trait_entities, after='ner')


The example above is rigid. It only check for PLAN entities with a preceding one-token adjective. This needs to be generalized to explicitly include any dependency relationehips (e.g., 'the awns are rough' in addition to 'rough awns')

In [17]:
plan_entities = [ent for ent in doc.ents if ent.label_ == "PLAN"]
for ent in plan_entities:
    # Because the entity is a spans, we need to use its root token. The head
    # is the syntactic governor of the person, e.g. the verb
    head = ent.root.head
    if head.lemma_ == "be":
        # Check if the children contain an adjectival complement (acomp)
        acomps = [token for token in head.children if token.dep_ == "acomp"]
        for acomp in acomps:
            print(acomp, ent, "is a TRAT (trait)")

white aleurone is a TRAT (trait)


Ah, now all we need to do is add this piece of logic to the earlier compound_trait_entities function we will add to the pipeline. This handles phrases where the adjective doesn't precede the noun it modifies, and the original code already handles the phrases where the adjective directly precedes the noun it modifies.